---
title: "$K$ Nearest Neighbors in Python"
subtitle: "Applied Machine Learning"
author: "Calvin"
  
jupyter: python3

execute:
    echo: true
    cache: true
    freeze: true  # never re-render during project render
---


## Agenda

1. Review of Homeworks
2. A human understanding of regression
3. Dinner break
5. Preprocessing and BoxCox
6. The $K$NN algorithm and the Confusion Matrix

# Homework

## HW1

- We need to work on writing quality.
- We need to work on RMSE intepretation.
- We need to work on using `summary` responsibly.
- We need to work on applying lecture topics to leading questions.
- We would benefit from use of the [`embed-resources`](https://quarto.org/docs/reference/formats/html.html) option in Quarto.

## [HW1 Sols Posted](../hws/wine_of_pnw_sol.html)

<iframe src="../hws/wine_of_pnw_sol.html" height="500" width="800" title="Iframe Example"></iframe> 

## HW2

- Think
- Pair
- Share

# Today

## Setup

## Import

-   Python base data stack

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## ML Library

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, make_scorer, cohen_kappa_score
import re

## Setup

-   `pyreadr` to read in an R dataset.


In [ ]:
# Local pickle as a performance concession.
# Same relative path as on GitHub
wine = pd.read_pickle("../../dat/wine.pickle")     
wine['lprice'] = np.log(wine['price'])

# Reporting Impact from Regressions

## Correlation

[*Credit:* Modern Dive](https://moderndive.com/5-regression.html)

[http://guessthecorrelation.com/](http://guessthecorrelation.com/) ...my high score is 72 (Jameson 122)

![](./images/correlation.png)

## Calculating correlation


In [ ]:
wine['price'].corr(wine['points']), wine['lprice'].corr(wine['points'])

## Exercise

1. Calculate the correlation between $\log$(price) and points... 
2. ...by variety...
3. ...for Oregon Chardonnay, Pinot Noir and Pinot Gris...
4. ...in the same expression!

## Solution

- Make a function...


In [ ]:
def var_cor(wine, var):
  df = wine[(wine['province'] == "Oregon") & 
            (wine['variety']  == var)]
  return df['lprice'].corr(df['points'])

## Eval


In [ ]:
vs = ("Chardonnay", "Pinot Noir", "Pinot Gris")
_ = [print(v, var_cor(wine, v)) for v in vs]

## Visualizing these different correlations


In [ ]:
wine = wine[(wine['province'] == "Oregon") & 
                     (wine['variety'].isin(vs))]
g = sns.FacetGrid(wine, col="variety", hue="variety")
g.map_dataframe(sns.regplot, x="points", y="lprice")

## Graphing residuals (bad)


In [ ]:
X,y  = wine[['points']], wine['price']
wine['residual'] = y - LinearRegression().fit(X, y).predict(X)
sns.scatterplot(x='points', y='residual', data=wine, alpha=0.3)

## Graphing residuals (good)

- Clearly should've made a function here.

In [ ]:
def resid_plot(df, x, y):
  X,y  = df[[x]], df[y]
  wine['residual'] = y - LinearRegression().fit(X, y).predict(X)
  return sns.scatterplot(x='points', y='residual', data=wine, alpha=0.3)

## Try it


In [ ]:
resid_plot(wine, "points", "lprice")

## Try it on year


In [ ]:
resid_plot(wine, "year", "lprice")

## Interpreting the coefficients

- Before examining many coefficients, a helper.


In [ ]:
def get_coef(df, x, y):
  X,y  = df[[x]], df[y]
  return (LinearRegression().fit(X,y)).coef_[0]

## % Change


In [ ]:
get_pct = lambda x : (np.exp(x)- 1) * 100

coef = get_coef(wine, "points", "lprice")
coef, get_pct(coef) 

- We logged the *dependent* variable (price)
  - A 1 point ratings increase = ~10% price
  - That is, a percent change in rating to an *absolute* change in the dependent variable. 
- $(e^x - 1)*100$

## Interpreting the coefficients


In [ ]:
coef = get_coef(wine, "year", "lprice")
coef, get_pct(coef)

- This is a *de facto* measure of inflation.

## Some Examples

::::{.columns}

:::{.column width=50%}


In [ ]:
_ = [print(x/10, get_pct(x/10)) for x in range(10)]

:::


:::{.column width=50%}


In [ ]:
_ = [print(x/100, get_pct(x/100)) for x in range(10)]

:::

::::

## $\log$ feature


In [ ]:
wine['lpoints'] = np.log(wine['points'])
coef = get_coef(wine, "lpoints", "price")
coef, get_pct(coef) 

- What does the *sign* (positive or negative) tell us?
- Was $\log$ appropriate here?

## LogLog (also elasticity)


In [ ]:
coef = get_coef(wine, "lpoints", "lprice")
coef

...a 1\% increase in ratings gives a ~9.2% increase in price on average

## Summary

- Only the dependent/response variable is log-transformed. 
  - Exponentiate the coefficient.
  - Subtract one from this number
  - Multiply by 100.
- This gives the percent increase (or decrease).

## Units

- Change is per one-unit increase in the independent variable. 
  - Here, independent is points.
  - Dependent is price.

## Example

- For every 1% increase in the independent variable...
  - Basically, one point
  - Our dependent variable increases by about 9.2%.
- A $30 bottle of wine scoring 90 would be worth $32.75 as a 91.

In [ ]:
30 * (1 + 9.2/100)

## Graphing points by variety

In [ ]:
sns.boxplot(x='variety', y='points', data=wine)

## Summary

In [ ]:
wine.groupby('variety')['points'].mean()

- What are the percentage differences here?

## Assumptions of linear regression

1. **L**inearity of relationship between variables
2. **I**ndependence of the residuals
3. **N**ormality of the residuals
4. **E**quality of variance of the residuals

## Linearity of relationship
[*Credit:* Modern Dive](https://moderndive.com/5-regression.html)

![](./images/non-linear.png)

What would the residuals look like?

## Independence

- Given our original model of 
$$
\log(\text{price})=m*\text{Points}+b
$$ 

- are there any problems with independence?
- How could we check?

## Normality

In [ ]:
X = wine[["points"]]
sns.histplot(y - LinearRegression().fit(X, wine["lprice"]).predict(X))

## No equality in the variance

[*Credit:* Modern Dive](https://moderndive.com/5-regression.html) (click)

![](./images/unequal-variance.png)

# Preprocessing and BoxCox

## Setup

- Pivot to pinot.
  - Say "$\pi^0$"


In [ ]:
wine = pd.read_pickle("../../dat/pinot_orig.pickle") 

## Preprocessing

- Box-Cox transformations use maximum likelihood estimate to estimate value for $\lambda$

$$
y(\lambda) = \frac{x^{\lambda}-1}{\lambda}
$$

- The goal is to make data seem more like a normal distribution.

## in R

- LaTeX

$$
y(\lambda) = \frac{x^{\lambda}-1}{\lambda}
$$

- Py

In [ ]:
y = lambda l : lambda x : (x ** l - 1) / l

## Examples

- When $\lambda=1$, there is no transformation

$$
y(1) = \frac{x^{\lambda}-1}{\lambda} = 
      \frac{x^{1}-1}{1} =
        x-1 \approx x
$$

$$
f = y(-1) \implies f(x) \approx x
$$

## Examples

- When $\lambda=0$, it is log transformed

$$
y(0) = \frac{x^{\lambda}-1}{\lambda} = 
      \frac{x^{0}-1}{0}
$$

$$
f = y(0) \implies f(x) \approx \log(x)
$$


- Zero is a special case, but using a little thing called "calculus" this sorta makes sense.
- Basically, negative infinity at 0, then increase slowly once positive.

## $\lambda = 0$


In [ ]:
[y(10**-l)(4) - np.log(4) for l in range(5)]

## Examples

$$
y(1) = \frac{x^{\lambda}-1}{\lambda} = 
      \frac{x^{.5}-1}{.5} =
      2\times(\sqrt{x}-1) \approx \sqrt{x}
$$

$$
f = y(.5) \implies f(x) \approx \sqrt{x}
$$


## Examples

- When $\lambda=-1$, it is an inverse

$$
y(1) = \frac{x^{\lambda}-1}{\lambda} = 
      \frac{x^{-1}-1}{-1} =
      \frac{x^{-1}}{-1}+\frac{-1}{-1} =
      \frac{1}{-x}-1 = -x
$$
$$
f = y(.5) \implies f(x) \approx -x
$$

## Just use the function

- Use `PowerTransformer`

In [ ]:
data = np.array([1, 2, 3, 4, 5]).reshape(-1, 1)

# Apply PowerTransformer (Box-Cox transformation)
pt = PowerTransformer(method='box-cox')
boxcox_result = pt.fit_transform(data)

# Print the result
print("Transformed data:", boxcox_result.flatten())
print("Lambda value:", pt.lambdas_)

## On Points


In [ ]:
pt.fit_transform(wine["points"].values.reshape(-1, 1))
print("Transformed data:", boxcox_result.flatten())
print("Lambda value:", pt.lambdas_)

## On Python

- I like the Python [boxcox documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.boxcox.html):

```{.haskell}
y = (x**lmbda - 1) / lmbda,  for lmbda != 0
    log(x),                  for lmbda = 0
```

# The $K$NN Algorithm

## Algorithm {.smaller}

1. Load the data
2. Initialize $K$ to your chosen number of neighbors
3. For each example in the data
  - Calculate the distance between the query example and the current example from the data.
  - Add the distance and the index of the example to an ordered collection
4. Sort the ordered collection of distances and indices from smallest to largest (in ascending order) by the distances
5. Pick the first $K$ entries from the sorted collection
6. Get the labels of the selected $K$ entries
7. If regression, return the mean of the $K$ labels
8. If classification, return the mode of the $K$ labels

## Basis

- We assume:
  - Existing datapoints in something we think of as a space
    - That is, probably two numerical value per point in a coordinate plane
    - Categorical is fine - think a Punnett square
  - Existing datapoints are labelled
    - Numerical or categorical still fine!
- To visualize, we will have a 2d space with color labels.

## Let's draw it

<a  title="Antti Ajanki AnAj, CC BY-SA 3.0 &lt;http://creativecommons.org/licenses/by-sa/3.0/&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:KnnClassification.svg"><img style="background-color:white;" width="50%" alt="KnnClassification" src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e7/KnnClassification.svg/256px-KnnClassification.svg.png?20101222011435"></a>

## Let's draw it 2


<a title="Paolo Bonfini, CC BY-SA 4.0 &lt;https://creativecommons.org/licenses/by-sa/4.0&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:KNN_decision_surface_animation.gif"><img width="100%" alt="KNN decision surface animation" src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/78/KNN_decision_surface_animation.gif/512px-KNN_decision_surface_animation.gif?20240718130312"></a>

## Let's draw it 3


![](https://media.geeksforgeeks.org/wp-content/uploads/20231207103856/KNN-Algorithm-(1).png)

[G4G](https://www.geeksforgeeks.org/k-nearest-neighbours/)

## Engineering some features

- Tasters, years, notes


In [ ]:
wine = pd.get_dummies(wine, columns=['year', 'taster_name'], drop_first=True)
wine.columns = [re.sub(r'\s+', '_', re.sub(r'-', '_', col.lower())) for col in wine.columns]
notes = ['cherry', 'chocolate', 'earth']
for note in notes:
    wine[f'note_{note}'] = wine['description'].str.contains(note, case=False, na=False)

wine = wine.drop('description', axis=1)

## Split


In [ ]:
train, test = train_test_split(wine, test_size=0.2, stratify=wine['province'], random_state=505)

## Confusion matrix

- Let's look at misses


In [ ]:
X_train, X_test = train.drop(columns=['province']), test.drop(columns=['province'])
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, train['province'])
y_pred = knn.predict(X_test)
conf_matrix_df = pd.DataFrame(confusion_matrix(test['province'], y_pred, labels=knn.classes_), index=knn.classes_, columns=knn.classes_)
sns.heatmap(conf_matrix_df, annot=True, fmt='d', cmap='Blues')

## Determine what dominates


In [ ]:
def omit(prefix):
  X_train, X_test = train.drop(columns=['province']), test.drop(columns=['province'])
  columns_to_drop = [col for col in X_train.columns if prefix in col]
  X_train = X_train.drop(columns=columns_to_drop)
  X_test = X_test.drop(columns=columns_to_drop)
  knn = KNeighborsClassifier(n_neighbors=5)
  knn.fit(X_train, train['province'])
  y_pred = knn.predict(X_test)
  conf_matrix_df = pd.DataFrame(confusion_matrix(test['province'], y_pred, labels=knn.classes_), index=knn.classes_, columns=knn.classes_)
  return sns.heatmap(conf_matrix_df, annot=True, fmt='d', cmap='Blues')

## Test

::::{.columns}

:::{.column width=33%}


In [ ]:
omit('year')

:::

:::{.column width=33%}


In [ ]:
omit('note')

:::

:::{.column width=33%}


In [ ]:
omit('taster')

:::

:::

## Drop it


In [ ]:
columns_to_drop = [col for col in X_train.columns if 'taster' in col]

train = train.drop(columns=columns_to_drop)

test = test.drop(columns=columns_to_drop)

## Tuning and subsampling


In [ ]:
X_train, X_test = train.drop(columns=['province']), test.drop(columns=['province'])
knn = KNeighborsClassifier()
param_grid = {'n_neighbors': range(1, 16)}
kappa_scorer = make_scorer(cohen_kappa_score)
grid_search = GridSearchCV(knn, param_grid, scoring=kappa_scorer)
grid_search.fit(X_train, train['province'])
cohen_kappa_score(test['province'], grid_search.best_estimator_.predict(X_test))

## Group modeling problem I

* Practice running different versions of the model
* Create some new features and...
* See if you can achieve a Kappa >= 0.5!